# 🧠 AI Platform Engine Learning Enhancement

This notebook enhances the learning capabilities of your AI platform using Google Colab's GPU/TPU resources.

## Features:
- Pattern recognition enhancement
- User behavior analysis
- Model fine-tuning
- Real-time learning optimization
- Multi-modal learning integration

## 1. Environment Setup & GPU Configuration

In [ ]:
# Check GPU availability and setup
import tensorflow as tf
import torch
import numpy as np
from datetime import datetime
import os

# Check for GPU
if tf.test.gpu_device_name():
    print(f'✅ GPU found: {tf.test.gpu_device_name()}')
else:
    print('❌ No GPU found. Using CPU.')

# PyTorch GPU check
print(f'\n🔥 PyTorch GPU: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU Device: {torch.cuda.get_device_name(0)}')
    print(f'Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB')

In [ ]:
# Install required packages
!pip install -q transformers datasets accelerate
!pip install -q sentence-transformers
!pip install -q scikit-learn pandas matplotlib seaborn
!pip install -q wandb  # For experiment tracking
!pip install -q optuna  # For hyperparameter optimization
!pip install -q redis  # For caching
!pip install -q fastapi uvicorn  # For API

print("✅ All dependencies installed")

## 2. Enhanced Learning Engine

In [ ]:
from typing import Dict, List, Tuple, Any, Optional
from dataclasses import dataclass, field
import json
from collections import defaultdict
import pickle

@dataclass
class LearningPattern:
    """Represents a learned pattern from user interactions"""
    pattern_id: str
    pattern_type: str  # 'prompt_improvement', 'rejection_recovery', 'style_preference'
    input_features: Dict[str, Any]
    output_features: Dict[str, Any]
    confidence: float
    frequency: int
    last_seen: datetime
    success_rate: float
    embeddings: Optional[np.ndarray] = None

@dataclass
class UserLearningProfile:
    """Advanced user-specific learning profile"""
    user_id: str
    learned_patterns: List[LearningPattern] = field(default_factory=list)
    preference_embeddings: Optional[np.ndarray] = None
    skill_progression: Dict[str, float] = field(default_factory=dict)
    interaction_history: List[Dict] = field(default_factory=list)
    personalization_vector: Optional[np.ndarray] = None
    
class EnhancedLearningEngine:
    """GPU-accelerated learning engine for pattern recognition and optimization"""
    
    def __init__(self, use_gpu: bool = True):
        self.use_gpu = use_gpu and torch.cuda.is_available()
        self.device = torch.device('cuda' if self.use_gpu else 'cpu')
        print(f"🚀 Learning Engine initialized on {self.device}")
        
        # Initialize models
        self._init_models()
        
        # Learning storage
        self.user_profiles: Dict[str, UserLearningProfile] = {}
        self.global_patterns: List[LearningPattern] = []
        self.pattern_embeddings = None
        
    def _init_models(self):
        """Initialize ML models for learning"""
        from sentence_transformers import SentenceTransformer
        
        # Sentence embedding model for semantic understanding
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
        if self.use_gpu:
            self.embedding_model = self.embedding_model.to(self.device)
        
        # Pattern matching model
        self.pattern_matcher = self._build_pattern_matcher()
        
        # Success prediction model
        self.success_predictor = self._build_success_predictor()
        
    def _build_pattern_matcher(self):
        """Build neural network for pattern matching"""
        import torch.nn as nn
        
        class PatternMatcher(nn.Module):
            def __init__(self, input_dim=384, hidden_dim=256, output_dim=128):
                super().__init__()
                self.encoder = nn.Sequential(
                    nn.Linear(input_dim, hidden_dim),
                    nn.ReLU(),
                    nn.Dropout(0.2),
                    nn.Linear(hidden_dim, hidden_dim),
                    nn.ReLU(),
                    nn.Linear(hidden_dim, output_dim)
                )
                
            def forward(self, x):
                return self.encoder(x)
        
        model = PatternMatcher().to(self.device)
        return model
    
    def _build_success_predictor(self):
        """Build model to predict optimization success"""
        import torch.nn as nn
        
        class SuccessPredictor(nn.Module):
            def __init__(self, input_dim=512, hidden_dim=256):
                super().__init__()
                self.predictor = nn.Sequential(
                    nn.Linear(input_dim, hidden_dim),
                    nn.ReLU(),
                    nn.Dropout(0.3),
                    nn.Linear(hidden_dim, 128),
                    nn.ReLU(),
                    nn.Linear(128, 1),
                    nn.Sigmoid()
                )
                
            def forward(self, x):
                return self.predictor(x)
        
        model = SuccessPredictor().to(self.device)
        return model
    
    def learn_from_interaction(self, user_id: str, interaction_data: Dict[str, Any]):
        """Learn from a user interaction"""
        # Get or create user profile
        if user_id not in self.user_profiles:
            self.user_profiles[user_id] = UserLearningProfile(user_id=user_id)
        
        profile = self.user_profiles[user_id]
        
        # Extract features
        input_embedding = self._embed_text(interaction_data.get('input', ''))
        output_embedding = self._embed_text(interaction_data.get('output', ''))
        
        # Create learning pattern
        pattern = LearningPattern(
            pattern_id=f"{user_id}_{datetime.now().timestamp()}",
            pattern_type=interaction_data.get('type', 'general'),
            input_features=self._extract_features(interaction_data['input']),
            output_features=self._extract_features(interaction_data['output']),
            confidence=interaction_data.get('confidence', 0.8),
            frequency=1,
            last_seen=datetime.now(),
            success_rate=interaction_data.get('success_rate', 0.85),
            embeddings=np.concatenate([input_embedding, output_embedding])
        )
        
        # Update profile
        profile.learned_patterns.append(pattern)
        profile.interaction_history.append(interaction_data)
        
        # Update skill progression
        self._update_skill_progression(profile, interaction_data)
        
        # Update personalization vector
        self._update_personalization_vector(profile)
        
        return pattern
    
    def _embed_text(self, text: str) -> np.ndarray:
        """Generate embeddings for text"""
        with torch.no_grad():
            embedding = self.embedding_model.encode(text, convert_to_tensor=True)
            return embedding.cpu().numpy()
    
    def _extract_features(self, text: str) -> Dict[str, Any]:
        """Extract linguistic and semantic features"""
        features = {
            'length': len(text.split()),
            'complexity': self._calculate_complexity(text),
            'sentiment': self._analyze_sentiment(text),
            'keywords': self._extract_keywords(text),
            'style': self._detect_style(text)
        }
        return features
    
    def _calculate_complexity(self, text: str) -> float:
        """Calculate text complexity score"""
        words = text.split()
        avg_word_length = np.mean([len(w) for w in words]) if words else 0
        unique_ratio = len(set(words)) / len(words) if words else 0
        return (avg_word_length / 10 + unique_ratio) / 2
    
    def _analyze_sentiment(self, text: str) -> Dict[str, float]:
        """Simple sentiment analysis"""
        # Placeholder - in production use proper sentiment model
        positive_words = ['good', 'great', 'excellent', 'amazing', 'wonderful']
        negative_words = ['bad', 'poor', 'terrible', 'awful', 'horrible']
        
        text_lower = text.lower()
        pos_score = sum(1 for word in positive_words if word in text_lower)
        neg_score = sum(1 for word in negative_words if word in text_lower)
        
        total = pos_score + neg_score + 1
        return {
            'positive': pos_score / total,
            'negative': neg_score / total,
            'neutral': 1 - (pos_score + neg_score) / total
        }
    
    def _extract_keywords(self, text: str) -> List[str]:
        """Extract key terms from text"""
        # Simple keyword extraction
        import re
        words = re.findall(r'\b\w{4,}\b', text.lower())
        word_freq = defaultdict(int)
        for word in words:
            word_freq[word] += 1
        
        # Return top 5 most frequent words
        return sorted(word_freq.keys(), key=word_freq.get, reverse=True)[:5]
    
    def _detect_style(self, text: str) -> str:
        """Detect writing style"""
        if len(text.split()) < 10:
            return 'concise'
        elif any(word in text.lower() for word in ['cinematic', 'artistic', 'dramatic']):
            return 'creative'
        elif any(word in text.lower() for word in ['analyze', 'evaluate', 'examine']):
            return 'analytical'
        else:
            return 'neutral'
    
    def _update_skill_progression(self, profile: UserLearningProfile, interaction: Dict):
        """Update user's skill progression"""
        skill_type = interaction.get('skill_type', 'general')
        success = interaction.get('success', True)
        
        if skill_type not in profile.skill_progression:
            profile.skill_progression[skill_type] = 0.5
        
        # Update with exponential moving average
        alpha = 0.1  # Learning rate
        current = profile.skill_progression[skill_type]
        profile.skill_progression[skill_type] = (1 - alpha) * current + alpha * (1.0 if success else 0.0)
    
    def _update_personalization_vector(self, profile: UserLearningProfile):
        """Update user's personalization vector"""
        if len(profile.learned_patterns) < 3:
            return
        
        # Aggregate pattern embeddings
        recent_patterns = profile.learned_patterns[-20:]  # Last 20 patterns
        embeddings = [p.embeddings for p in recent_patterns if p.embeddings is not None]
        
        if embeddings:
            # Average embeddings weighted by success rate and recency
            weights = []
            for i, pattern in enumerate(recent_patterns):
                recency_weight = (i + 1) / len(recent_patterns)
                success_weight = pattern.success_rate
                weights.append(recency_weight * success_weight)
            
            weights = np.array(weights) / np.sum(weights)
            weighted_embeddings = np.average(embeddings, axis=0, weights=weights[:len(embeddings)])
            profile.personalization_vector = weighted_embeddings

# Initialize the learning engine
learning_engine = EnhancedLearningEngine()
print("✅ Enhanced Learning Engine ready!")

## 3. Advanced Pattern Mining & Recognition

In [ ]:
class PatternMiner:
    """GPU-accelerated pattern mining from user interactions"""
    
    def __init__(self, learning_engine: EnhancedLearningEngine):
        self.engine = learning_engine
        self.mined_patterns = []
        
    def mine_frequent_patterns(self, min_support: float = 0.1) -> List[Dict]:
        """Mine frequent patterns across all users"""
        all_patterns = []
        
        # Collect all patterns
        for profile in self.engine.user_profiles.values():
            all_patterns.extend(profile.learned_patterns)
        
        if not all_patterns:
            return []
        
        # Convert to embeddings matrix
        embeddings = np.array([p.embeddings for p in all_patterns if p.embeddings is not None])
        
        if len(embeddings) == 0:
            return []
        
        # Use GPU for clustering
        embeddings_tensor = torch.tensor(embeddings, device=self.engine.device)
        
        # Perform clustering to find pattern groups
        from sklearn.cluster import DBSCAN
        
        # Move to CPU for sklearn
        embeddings_cpu = embeddings_tensor.cpu().numpy()
        
        clustering = DBSCAN(eps=0.3, min_samples=max(2, int(len(embeddings) * min_support)))
        labels = clustering.fit_predict(embeddings_cpu)
        
        # Extract pattern groups
        pattern_groups = defaultdict(list)
        for i, label in enumerate(labels):
            if label != -1:  # Not noise
                pattern_groups[label].append(all_patterns[i])
        
        # Analyze each group
        mined_patterns = []
        for group_id, patterns in pattern_groups.items():
            if len(patterns) >= max(2, int(len(all_patterns) * min_support)):
                pattern_info = self._analyze_pattern_group(patterns)
                pattern_info['group_id'] = group_id
                pattern_info['support'] = len(patterns) / len(all_patterns)
                mined_patterns.append(pattern_info)
        
        self.mined_patterns = mined_patterns
        return mined_patterns
    
    def _analyze_pattern_group(self, patterns: List[LearningPattern]) -> Dict:
        """Analyze a group of similar patterns"""
        # Calculate group statistics
        avg_confidence = np.mean([p.confidence for p in patterns])
        avg_success = np.mean([p.success_rate for p in patterns])
        
        # Find common features
        input_features = defaultdict(list)
        output_features = defaultdict(list)
        
        for pattern in patterns:
            for key, value in pattern.input_features.items():
                if isinstance(value, (int, float)):
                    input_features[key].append(value)
            for key, value in pattern.output_features.items():
                if isinstance(value, (int, float)):
                    output_features[key].append(value)
        
        # Calculate feature statistics
        input_stats = {k: {'mean': np.mean(v), 'std': np.std(v)} 
                      for k, v in input_features.items() if v}
        output_stats = {k: {'mean': np.mean(v), 'std': np.std(v)} 
                       for k, v in output_features.items() if v}
        
        # Find most common pattern type
        pattern_types = [p.pattern_type for p in patterns]
        most_common_type = max(set(pattern_types), key=pattern_types.count)
        
        return {
            'pattern_count': len(patterns),
            'avg_confidence': avg_confidence,
            'avg_success_rate': avg_success,
            'dominant_type': most_common_type,
            'input_feature_stats': input_stats,
            'output_feature_stats': output_stats,
            'sample_patterns': patterns[:3]  # Keep some examples
        }
    
    def find_optimization_rules(self) -> List[Dict]:
        """Extract optimization rules from mined patterns"""
        rules = []
        
        for pattern_group in self.mined_patterns:
            if pattern_group['avg_success_rate'] > 0.8:  # High success patterns
                rule = {
                    'rule_id': f"rule_{pattern_group['group_id']}",
                    'condition': self._extract_condition(pattern_group),
                    'action': self._extract_action(pattern_group),
                    'confidence': pattern_group['avg_confidence'],
                    'support': pattern_group['support'],
                    'success_rate': pattern_group['avg_success_rate']
                }
                rules.append(rule)
        
        return sorted(rules, key=lambda r: r['confidence'] * r['support'], reverse=True)
    
    def _extract_condition(self, pattern_group: Dict) -> Dict:
        """Extract rule conditions from pattern group"""
        conditions = {}
        
        # Input feature conditions
        for feature, stats in pattern_group['input_feature_stats'].items():
            if stats['std'] < stats['mean'] * 0.2:  # Low variance
                conditions[f'input_{feature}'] = {
                    'type': 'range',
                    'min': stats['mean'] - stats['std'],
                    'max': stats['mean'] + stats['std']
                }
        
        conditions['pattern_type'] = pattern_group['dominant_type']
        return conditions
    
    def _extract_action(self, pattern_group: Dict) -> Dict:
        """Extract rule actions from pattern group"""
        actions = {}
        
        # Output feature actions
        for feature, stats in pattern_group['output_feature_stats'].items():
            actions[f'set_{feature}'] = stats['mean']
        
        # Add sample transformations
        if pattern_group['sample_patterns']:
            sample = pattern_group['sample_patterns'][0]
            actions['sample_transformation'] = {
                'from': sample.input_features,
                'to': sample.output_features
            }
        
        return actions

# Create pattern miner
pattern_miner = PatternMiner(learning_engine)
print("✅ Pattern Miner initialized")

## 4. Real-time Learning & Adaptation

In [ ]:
class RealtimeAdapter:
    """Real-time adaptation based on learned patterns"""
    
    def __init__(self, learning_engine: EnhancedLearningEngine, pattern_miner: PatternMiner):
        self.engine = learning_engine
        self.miner = pattern_miner
        self.adaptation_cache = {}
        self.performance_history = []
        
    def adapt_optimization(self, user_id: str, prompt: str, context: Dict[str, Any]) -> Dict:
        """Adapt optimization strategy based on learned patterns"""
        # Get user profile
        profile = self.engine.user_profiles.get(user_id)
        
        # Generate prompt embedding
        prompt_embedding = self.engine._embed_text(prompt)
        
        # Find similar successful patterns
        similar_patterns = self._find_similar_patterns(prompt_embedding, profile)
        
        # Apply learned optimizations
        optimization_strategy = self._build_optimization_strategy(similar_patterns, context)
        
        # Predict success probability
        success_probability = self._predict_success(prompt_embedding, optimization_strategy)
        
        # Cache the adaptation
        cache_key = f"{user_id}_{hash(prompt)}"
        self.adaptation_cache[cache_key] = {
            'strategy': optimization_strategy,
            'probability': success_probability,
            'timestamp': datetime.now()
        }
        
        return {
            'adapted_strategy': optimization_strategy,
            'success_probability': float(success_probability),
            'similar_patterns_found': len(similar_patterns),
            'personalization_applied': profile is not None,
            'confidence': self._calculate_confidence(similar_patterns)
        }
    
    def _find_similar_patterns(self, embedding: np.ndarray, profile: Optional[UserLearningProfile]) -> List[LearningPattern]:
        """Find patterns similar to current input"""
        similar_patterns = []
        
        # Search in user's patterns first
        if profile and profile.learned_patterns:
            user_patterns = [(p, self._calculate_similarity(embedding, p.embeddings)) 
                           for p in profile.learned_patterns if p.embeddings is not None]
            user_patterns.sort(key=lambda x: x[1], reverse=True)
            similar_patterns.extend([p for p, sim in user_patterns[:5] if sim > 0.7])
        
        # Search in global patterns
        if self.miner.mined_patterns:
            for pattern_group in self.miner.mined_patterns:
                for sample_pattern in pattern_group['sample_patterns']:
                    if sample_pattern.embeddings is not None:
                        sim = self._calculate_similarity(embedding, sample_pattern.embeddings[:len(embedding)])
                        if sim > 0.75:
                            similar_patterns.append(sample_pattern)
        
        return similar_patterns[:10]  # Top 10 most similar
    
    def _calculate_similarity(self, embedding1: np.ndarray, embedding2: np.ndarray) -> float:
        """Calculate cosine similarity between embeddings"""
        if embedding2 is None or len(embedding1) != len(embedding2):
            return 0.0
        
        dot_product = np.dot(embedding1, embedding2)
        norm1 = np.linalg.norm(embedding1)
        norm2 = np.linalg.norm(embedding2)
        
        if norm1 == 0 or norm2 == 0:
            return 0.0
        
        return dot_product / (norm1 * norm2)
    
    def _build_optimization_strategy(self, patterns: List[LearningPattern], context: Dict) -> Dict:
        """Build optimization strategy from similar patterns"""
        if not patterns:
            return self._default_strategy()
        
        # Aggregate strategies from successful patterns
        strategy = {
            'approach': 'adaptive',
            'techniques': [],
            'parameters': {},
            'focus_areas': []
        }
        
        # Analyze pattern types
        pattern_types = [p.pattern_type for p in patterns]
        type_counts = defaultdict(int)
        for ptype in pattern_types:
            type_counts[ptype] += 1
        
        # Set primary approach based on most common pattern type
        primary_type = max(type_counts.keys(), key=type_counts.get)
        strategy['approach'] = self._map_pattern_type_to_approach(primary_type)
        
        # Extract techniques from successful patterns
        for pattern in patterns:
            if pattern.success_rate > 0.8:
                technique = self._extract_technique(pattern)
                if technique and technique not in strategy['techniques']:
                    strategy['techniques'].append(technique)
        
        # Set parameters based on pattern features
        strategy['parameters'] = self._aggregate_parameters(patterns)
        
        # Identify focus areas
        strategy['focus_areas'] = self._identify_focus_areas(patterns)
        
        return strategy
    
    def _default_strategy(self) -> Dict:
        """Default optimization strategy"""
        return {
            'approach': 'balanced',
            'techniques': ['enhance_clarity', 'add_context', 'improve_specificity'],
            'parameters': {
                'enhancement_level': 0.7,
                'context_depth': 'moderate',
                'safety_check': True
            },
            'focus_areas': ['clarity', 'relevance', 'safety']
        }
    
    def _map_pattern_type_to_approach(self, pattern_type: str) -> str:
        """Map pattern type to optimization approach"""
        mapping = {
            'prompt_improvement': 'enhancement',
            'rejection_recovery': 'safety_first',
            'style_preference': 'style_adaptive',
            'context_expansion': 'context_rich',
            'creative_enhancement': 'creative'
        }
        return mapping.get(pattern_type, 'balanced')
    
    def _extract_technique(self, pattern: LearningPattern) -> Optional[str]:
        """Extract optimization technique from pattern"""
        # Analyze input/output differences
        input_complexity = pattern.input_features.get('complexity', 0)
        output_complexity = pattern.output_features.get('complexity', 0)
        
        if output_complexity > input_complexity * 1.2:
            return 'expand_details'
        elif pattern.output_features.get('style') == 'creative':
            return 'add_creativity'
        elif len(pattern.output_features.get('keywords', [])) > len(pattern.input_features.get('keywords', [])):
            return 'enrich_vocabulary'
        
        return None
    
    def _aggregate_parameters(self, patterns: List[LearningPattern]) -> Dict:
        """Aggregate optimization parameters from patterns"""
        params = {
            'enhancement_level': 0.7,
            'creativity_boost': 0.0,
            'safety_threshold': 0.8,
            'context_depth': 'moderate'
        }
        
        # Calculate average success-weighted parameters
        if patterns:
            avg_complexity = np.mean([p.output_features.get('complexity', 0.5) for p in patterns])
            params['enhancement_level'] = min(0.9, avg_complexity * 1.2)
            
            creative_patterns = [p for p in patterns if p.output_features.get('style') == 'creative']
            if len(creative_patterns) > len(patterns) * 0.3:
                params['creativity_boost'] = 0.3
        
        return params
    
    def _identify_focus_areas(self, patterns: List[LearningPattern]) -> List[str]:
        """Identify areas to focus on based on patterns"""
        focus_areas = set()
        
        for pattern in patterns:
            # Check for consistent improvements
            if pattern.output_features.get('clarity', 0) > pattern.input_features.get('clarity', 0):
                focus_areas.add('clarity')
            if pattern.pattern_type == 'style_preference':
                focus_areas.add('style')
            if pattern.pattern_type == 'rejection_recovery':
                focus_areas.add('safety')
        
        return list(focus_areas) or ['general']
    
    def _predict_success(self, embedding: np.ndarray, strategy: Dict) -> float:
        """Predict success probability using neural network"""
        # Prepare input features
        strategy_features = self._encode_strategy(strategy)
        
        # Combine embeddings and strategy
        combined_features = np.concatenate([embedding, strategy_features])
        
        # Ensure correct input dimension
        if len(combined_features) < 512:
            combined_features = np.pad(combined_features, (0, 512 - len(combined_features)))
        elif len(combined_features) > 512:
            combined_features = combined_features[:512]
        
        # Convert to tensor
        input_tensor = torch.tensor(combined_features, dtype=torch.float32).unsqueeze(0).to(self.engine.device)
        
        # Predict
        with torch.no_grad():
            self.engine.success_predictor.eval()
            prediction = self.engine.success_predictor(input_tensor)
            return prediction.item()
    
    def _encode_strategy(self, strategy: Dict) -> np.ndarray:
        """Encode strategy as feature vector"""
        # Simple encoding - in production use more sophisticated encoding
        features = []
        
        # Approach encoding (one-hot)
        approaches = ['adaptive', 'enhancement', 'safety_first', 'creative', 'balanced']
        approach_vec = [1.0 if strategy['approach'] == a else 0.0 for a in approaches]
        features.extend(approach_vec)
        
        # Technique count
        features.append(len(strategy['techniques']) / 10.0)
        
        # Parameter encoding
        features.append(strategy['parameters'].get('enhancement_level', 0.5))
        features.append(strategy['parameters'].get('creativity_boost', 0.0))
        features.append(strategy['parameters'].get('safety_threshold', 0.8))
        
        # Focus area count
        features.append(len(strategy['focus_areas']) / 5.0)
        
        return np.array(features)
    
    def _calculate_confidence(self, patterns: List[LearningPattern]) -> float:
        """Calculate confidence score based on pattern evidence"""
        if not patterns:
            return 0.5
        
        # Factors: number of patterns, average success rate, recency
        pattern_count_score = min(1.0, len(patterns) / 10.0)
        avg_success = np.mean([p.success_rate for p in patterns])
        
        # Recency score
        now = datetime.now()
        recency_scores = []
        for p in patterns:
            days_old = (now - p.last_seen).days
            recency_scores.append(max(0, 1 - days_old / 30))  # Decay over 30 days
        avg_recency = np.mean(recency_scores)
        
        # Weighted combination
        confidence = 0.3 * pattern_count_score + 0.5 * avg_success + 0.2 * avg_recency
        return float(confidence)

# Create realtime adapter
realtime_adapter = RealtimeAdapter(learning_engine, pattern_miner)
print("✅ Realtime Adapter ready")

## 5. API Integration with Your Platform

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Optional
import uvicorn
import nest_asyncio

# Allow nested asyncio for Colab
nest_asyncio.apply()

# API Models
class LearningInteraction(BaseModel):
    user_id: str
    input_prompt: str
    output_prompt: str
    interaction_type: str = 'general'
    success: bool = True
    confidence: float = 0.8
    metadata: Optional[Dict] = None

class OptimizationRequest(BaseModel):
    user_id: str
    prompt: str
    context: Optional[Dict] = None

class PatternMiningRequest(BaseModel):
    min_support: float = 0.1
    user_filter: Optional[str] = None

# Create FastAPI app
app = FastAPI(title="AI Platform Learning Engine", version="1.0.0")

@app.get("/")
async def root():
    return {
        "message": "AI Platform Learning Engine API",
        "endpoints": [
            "/learn",
            "/adapt",
            "/mine-patterns",
            "/user-profile/{user_id}",
            "/learning-stats"
        ]
    }

@app.post("/learn")
async def learn_from_interaction(interaction: LearningInteraction):
    """Record and learn from a user interaction"""
    try:
        # Prepare interaction data
        interaction_data = {
            'input': interaction.input_prompt,
            'output': interaction.output_prompt,
            'type': interaction.interaction_type,
            'success': interaction.success,
            'confidence': interaction.confidence,
            'success_rate': 1.0 if interaction.success else 0.0
        }
        
        if interaction.metadata:
            interaction_data.update(interaction.metadata)
        
        # Learn from interaction
        pattern = learning_engine.learn_from_interaction(
            interaction.user_id,
            interaction_data
        )
        
        return {
            "status": "success",
            "pattern_id": pattern.pattern_id,
            "message": "Interaction learned successfully"
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/adapt")
async def adapt_optimization(request: OptimizationRequest):
    """Get adapted optimization strategy based on learning"""
    try:
        result = realtime_adapter.adapt_optimization(
            request.user_id,
            request.prompt,
            request.context or {}
        )
        
        return {
            "status": "success",
            "adaptation": result
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/mine-patterns")
async def mine_patterns(request: PatternMiningRequest):
    """Mine patterns from all interactions"""
    try:
        patterns = pattern_miner.mine_frequent_patterns(request.min_support)
        rules = pattern_miner.find_optimization_rules()
        
        return {
            "status": "success",
            "patterns_found": len(patterns),
            "rules_extracted": len(rules),
            "patterns": patterns[:10],  # Top 10
            "rules": rules[:5]  # Top 5 rules
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/user-profile/{user_id}")
async def get_user_profile(user_id: str):
    """Get user learning profile"""
    profile = learning_engine.user_profiles.get(user_id)
    
    if not profile:
        raise HTTPException(status_code=404, detail="User profile not found")
    
    return {
        "user_id": profile.user_id,
        "pattern_count": len(profile.learned_patterns),
        "skill_progression": profile.skill_progression,
        "interaction_count": len(profile.interaction_history),
        "has_personalization": profile.personalization_vector is not None
    }

@app.get("/learning-stats")
async def get_learning_stats():
    """Get overall learning statistics"""
    total_users = len(learning_engine.user_profiles)
    total_patterns = sum(len(p.learned_patterns) for p in learning_engine.user_profiles.values())
    
    return {
        "total_users": total_users,
        "total_patterns": total_patterns,
        "mined_pattern_groups": len(pattern_miner.mined_patterns),
        "gpu_enabled": learning_engine.use_gpu,
        "device": str(learning_engine.device)
    }

# Function to run the server
def run_api_server(port=8000):
    """Run the API server"""
    config = uvicorn.Config(app, host="0.0.0.0", port=port, log_level="info")
    server = uvicorn.Server(config)
    server.run()

print("✅ API server defined. Run run_api_server() to start.")

## 6. Demo: Testing the Learning Engine

In [ ]:
# Demo: Simulate learning from interactions
print("🎯 Demo: Testing Learning Engine\n")

# Simulate some user interactions
demo_interactions = [
    {
        "user_id": "user123",
        "input": "elderly man and young woman in bedroom",
        "output": "An intergenerational mentorship scene in a well-lit study, featuring a distinguished elderly professor and his young protégé engaged in academic discussion",
        "type": "rejection_recovery",
        "success": True,
        "confidence": 0.92
    },
    {
        "user_id": "user123",
        "input": "romantic scene between couple",
        "output": "A cinematic portrayal of deep emotional connection between two souls, captured in golden hour lighting with artistic composition emphasizing their profound bond",
        "type": "style_preference",
        "success": True,
        "confidence": 0.88
    },
    {
        "user_id": "user456",
        "input": "futuristic city at night",
        "output": "A breathtaking cyberpunk metropolis illuminated by neon lights, with flying vehicles weaving between towering skyscrapers under a starlit sky, rendered in cinematic detail",
        "type": "creative_enhancement",
        "success": True,
        "confidence": 0.95
    }
]

# Learn from interactions
for interaction in demo_interactions:
    pattern = learning_engine.learn_from_interaction(
        interaction["user_id"],
        interaction
    )
    print(f"✅ Learned pattern: {pattern.pattern_type} (confidence: {pattern.confidence:.2f})")

# Mine patterns
print("\n🔍 Mining patterns...")
mined_patterns = pattern_miner.mine_frequent_patterns(min_support=0.1)
print(f"Found {len(mined_patterns)} pattern groups")

# Extract rules
rules = pattern_miner.find_optimization_rules()
print(f"Extracted {len(rules)} optimization rules")

# Test adaptation
print("\n🎯 Testing real-time adaptation...")
test_prompt = "old man and young girl talking"
adaptation = realtime_adapter.adapt_optimization("user123", test_prompt, {})

print(f"\nAdaptation Result:")
print(f"  Strategy: {adaptation['adapted_strategy']['approach']}")
print(f"  Success Probability: {adaptation['success_probability']:.2%}")
print(f"  Confidence: {adaptation['confidence']:.2%}")
print(f"  Techniques: {', '.join(adaptation['adapted_strategy']['techniques'][:3])}")

# Show user profile
print("\n👤 User Profile Summary:")
profile = learning_engine.user_profiles.get("user123")
if profile:
    print(f"  Patterns Learned: {len(profile.learned_patterns)}")
    print(f"  Skills: {list(profile.skill_progression.keys())}")
    print(f"  Has Personalization: {profile.personalization_vector is not None}")

## 7. Integration with Your Flask App

In [ ]:
# Code to integrate with your Flask app
print("📋 Integration Code for Your Flask App:\n")

integration_code = '''
# Add to your Flask app.py:

import requests
from functools import wraps

# Configuration
COLAB_LEARNING_API = "https://YOUR_NGROK_URL"  # Replace with your ngrok URL

def with_learning(f):
    """Decorator to add learning capability to endpoints"""
    @wraps(f)
    def decorated_function(*args, **kwargs):
        # Execute original function
        result = f(*args, **kwargs)
        
        # Extract learning data if successful
        if result.get_json() and result.get_json().get('success'):
            try:
                data = request.get_json()
                response = result.get_json()
                
                # Send to learning engine
                learning_data = {
                    "user_id": g.get('user_id', 'anonymous'),
                    "input_prompt": data.get('prompt', ''),
                    "output_prompt": response.get('optimized', ''),
                    "interaction_type": data.get('type', 'optimization'),
                    "success": True,
                    "confidence": response.get('confidence', 0.8)
                }
                
                # Async call to learning API
                requests.post(f"{COLAB_LEARNING_API}/learn", json=learning_data, timeout=1)
            except:
                pass  # Don't break main flow if learning fails
        
        return result
    return decorated_function

@app.route('/api/optimize-with-learning', methods=['POST'])
@secure_endpoint
@with_learning
def optimize_with_learning():
    """Optimization endpoint with learning capability"""
    data = request.get_json()
    prompt = data.get('prompt', '')
    user_id = g.get('user_id', 'anonymous')
    
    # Get adapted strategy from learning engine
    try:
        adaptation_response = requests.post(
            f"{COLAB_LEARNING_API}/adapt",
            json={
                "user_id": user_id,
                "prompt": prompt,
                "context": {"source": "web_app"}
            },
            timeout=2
        )
        
        if adaptation_response.status_code == 200:
            adaptation = adaptation_response.json()['adaptation']
            # Use adapted strategy for optimization
            # ... your optimization logic here ...
    except:
        # Fallback to default optimization
        pass
    
    # Continue with normal optimization
    return unified_optimize()
'''

print(integration_code)

# Save integration code
with open('/content/flask_integration.py', 'w') as f:
    f.write(integration_code)
print("\n✅ Integration code saved to /content/flask_integration.py")

## 8. Deploy Learning API with ngrok

In [ ]:
# Install and setup ngrok for external access
!pip install -q pyngrok

from pyngrok import ngrok
import threading

# Start API server in background
def start_server():
    run_api_server(port=8000)

# Run server in thread
server_thread = threading.Thread(target=start_server, daemon=True)
server_thread.start()

# Give server time to start
import time
time.sleep(3)

# Create ngrok tunnel
public_url = ngrok.connect(8000)
print(f"\n🌐 Learning Engine API available at: {public_url}")
print(f"\n📋 Use this URL in your Flask app: {public_url}")
print("\n✅ Learning Engine is running and accessible from anywhere!")

# Test the API
import requests
test_response = requests.get(f"{public_url}/")
print(f"\n🧪 API Test: {test_response.json()}")

## 9. Learning Analytics & Visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Create learning analytics dashboard
def plot_learning_analytics():
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('AI Platform Learning Analytics', fontsize=16)
    
    # 1. Pattern Distribution
    ax1 = axes[0, 0]
    pattern_types = defaultdict(int)
    for profile in learning_engine.user_profiles.values():
        for pattern in profile.learned_patterns:
            pattern_types[pattern.pattern_type] += 1
    
    if pattern_types:
        pd.Series(pattern_types).plot(kind='bar', ax=ax1, color='skyblue')
        ax1.set_title('Pattern Type Distribution')
        ax1.set_xlabel('Pattern Type')
        ax1.set_ylabel('Count')
    
    # 2. Success Rate Over Time
    ax2 = axes[0, 1]
    success_rates = []
    for profile in learning_engine.user_profiles.values():
        for pattern in profile.learned_patterns:
            success_rates.append({
                'time': pattern.last_seen,
                'success_rate': pattern.success_rate
            })
    
    if success_rates:
        df_success = pd.DataFrame(success_rates)
        df_success.set_index('time')['success_rate'].plot(ax=ax2, marker='o')
        ax2.set_title('Success Rate Trend')
        ax2.set_ylabel('Success Rate')
    
    # 3. User Skill Progression
    ax3 = axes[1, 0]
    skill_data = []
    for user_id, profile in learning_engine.user_profiles.items():
        for skill, level in profile.skill_progression.items():
            skill_data.append({
                'user': user_id,
                'skill': skill,
                'level': level
            })
    
    if skill_data:
        df_skills = pd.DataFrame(skill_data)
        pivot_skills = df_skills.pivot(index='user', columns='skill', values='level')
        sns.heatmap(pivot_skills, ax=ax3, cmap='YlOrRd', annot=True, fmt='.2f')
        ax3.set_title('User Skill Levels')
    
    # 4. Pattern Confidence Distribution
    ax4 = axes[1, 1]
    confidences = []
    for profile in learning_engine.user_profiles.values():
        confidences.extend([p.confidence for p in profile.learned_patterns])
    
    if confidences:
        ax4.hist(confidences, bins=20, color='lightgreen', edgecolor='black')
        ax4.set_title('Pattern Confidence Distribution')
        ax4.set_xlabel('Confidence')
        ax4.set_ylabel('Frequency')
    
    plt.tight_layout()
    plt.show()

# Generate analytics
plot_learning_analytics()

# Summary statistics
print("\n📊 Learning Engine Statistics:")
print(f"Total Users: {len(learning_engine.user_profiles)}")
print(f"Total Patterns Learned: {sum(len(p.learned_patterns) for p in learning_engine.user_profiles.values())}")
print(f"Pattern Groups Mined: {len(pattern_miner.mined_patterns)}")
print(f"GPU Acceleration: {'Enabled' if learning_engine.use_gpu else 'Disabled'}")

## 10. Save & Export Learning Models

In [ ]:
import pickle
import json
from datetime import datetime

# Create export directory
export_dir = "/content/learning_engine_export"
os.makedirs(export_dir, exist_ok=True)

# Save learning engine state
print("💾 Saving learning engine state...")

# 1. Save user profiles
profiles_data = {}
for user_id, profile in learning_engine.user_profiles.items():
    profiles_data[user_id] = {
        'pattern_count': len(profile.learned_patterns),
        'skill_progression': profile.skill_progression,
        'interaction_count': len(profile.interaction_history)
    }

with open(f"{export_dir}/user_profiles.json", 'w') as f:
    json.dump(profiles_data, f, indent=2)
print("✅ User profiles saved")

# 2. Save mined patterns
with open(f"{export_dir}/mined_patterns.pkl", 'wb') as f:
    pickle.dump(pattern_miner.mined_patterns, f)
print("✅ Mined patterns saved")

# 3. Save model weights
torch.save({
    'pattern_matcher': learning_engine.pattern_matcher.state_dict(),
    'success_predictor': learning_engine.success_predictor.state_dict(),
    'timestamp': datetime.now().isoformat()
}, f"{export_dir}/model_weights.pt")
print("✅ Model weights saved")

# 4. Create deployment package
deployment_code = '''
# Deployment code for your server
# This loads the trained models and provides learning capability

import torch
import pickle
import json

class LearningSingleton:
    _instance = None
    
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
            cls._instance.initialized = False
        return cls._instance
    
    def initialize(self, model_path):
        if not self.initialized:
            # Load models and data
            self.load_models(model_path)
            self.initialized = True
    
    def load_models(self, path):
        # Implementation to load saved models
        pass

# Usage in Flask:
learning = LearningSingleton()
learning.initialize('/path/to/models')
'''

with open(f"{export_dir}/deployment.py", 'w') as f:
    f.write(deployment_code)

# Create zip archive
!cd /content && zip -r learning_engine_export.zip learning_engine_export/

print("\n✅ Learning engine exported successfully!")
print(f"📦 Download: /content/learning_engine_export.zip")
print("\n🚀 Next steps:")
print("1. Download the export package")
print("2. Integrate with your Flask app using the provided code")
print("3. The learning engine will continuously improve your platform!")